# Mitigate Climate Change

This notebook outlines the general workflow for the data within the [Mitigate Climate Change](https://oceancentral.org/track/mitigate-climate-change) page of the Ocean Central website.

## Utils functions and globals used for making all figures

In [ ]:
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import cartopy.crs as ccrs
from shapely.geometry import box
import rioxarray
import re

from rasterio.features import geometry_mask
from scipy.stats import linregress
from tqdm import tqdm

# Open the biodiversity priority areas based on Zhao et al. 2020 (https://www.sciencedirect.com/science/article/abs/pii/S0006320719312182?via%3Dihub)
masked_data = rioxarray.open_rasterio('../Data/masked_top_30_percent_over_water.tif')

# Set the CRS for masked_data if it's not already set
if 'crs' not in masked_data.attrs:
    masked_data.rio.write_crs('EPSG:4326', inplace=True)

# Load SST dataset and EEZ shapefile
seas_shapefile_path = '../Data/World_Seas_IHO_v3/World_Seas_IHO_v3.shp'
SEAS_DF = gpd.read_file(seas_shapefile_path)

# Calculate linear trend and p-value for each grid point
def calculate_trend_and_significance(x):
    if np.isnan(x).all():
        return np.nan, np.nan, np.nan
    else:
        slope, intercept, _, p_value, _ = stats.linregress(range(len(x)), x)
        return slope, intercept, p_value

# Calculate the trend and significance of the trend at each pixel in an xarray dataset
def calculate_trend_df(climate_df):
    df_mean = climate_df.groupby('time.year').mean()
    
    # Apply the trend and p-value calculation to the entire dataset
    results = xr.apply_ufunc(
        calculate_trend_and_significance,
        df_mean,
        input_core_dims=[['year']],
        vectorize=True,
        output_core_dims=[[], [], []],
        output_dtypes=[float, float, float]
    )
    
    # Extract the trend and p-value into separate DataArrays
    trends_da = xr.DataArray(results[0], coords=df_mean.isel(year=0).coords, name='trend')
    pvalues_da = xr.DataArray(results[2], coords=df_mean.isel(year=0).coords, name='p_value')
    
    # Create a significance mask where p-value < 0.05
    significant_da = xr.DataArray((pvalues_da < 0.05), coords=pvalues_da.coords, name='significant')
    
    # Combine trend, p-value, and significance mask into a single dataset
    trend_significance_ds = xr.Dataset({
        'trend': trends_da,
        'p_value': pvalues_da,
        'significant': significant_da
    })
    
    # Set the CRS for the trends dataset to match the EEZ CRS
    trend_significance_ds = trend_significance_ds.rio.write_crs("epsg:4326")
    return trend_significance_ds

# Calculate area-weighted trend, significance for each sea/ocean area
def area_trend(trend_significance_ds, SEAS_DF=SEAS_DF):
    # Iterate over each sea/ocean area and calculate the area-weighted trend and significant area percentage
    area_weighted_trends = []
    
    # Check if 'lat' and 'lon' are in the dataset, otherwise check for 'latitude' and 'longitude'
    if 'lat' in trend_significance_ds.dims and 'lon' in trend_significance_ds.dims:
        trend_significance_ds = trend_significance_ds.rename({'lat': 'y', 'lon': 'x'})
    elif 'latitude' in trend_significance_ds.dims and 'longitude' in trend_significance_ds.dims:
        trend_significance_ds = trend_significance_ds.rename({'latitude': 'y', 'longitude': 'x'})

    # Interpolate biodiversity priority areas to the same resolution as the climate data
    masked_data_interp = masked_data.interp(
        x=trend_significance_ds['x'],
        y=trend_significance_ds['y'],
        method='nearest'
    )

    # Calculate the area for each grid cell (assumes lat/lon grid)
    lat = trend_significance_ds['y'].values
    lon = trend_significance_ds['x'].values
    
    # Calculate grid cell area using Haversine formula or by approximation
    lat_rad = np.deg2rad(lat)
    lon_rad = np.deg2rad(lon)
    
    # Earth radius in kilometers
    R = 6371
    dlat = np.gradient(lat_rad)
    dlon = np.gradient(lon_rad)
    
    # Approximate area calculation
    cell_areas = (R**2 * np.outer(np.sin(dlat), dlon)) * np.cos(lat_rad[:, None])
    
    for i, row in SEAS_DF.iterrows():
        try:
            region_name = row['NAME']
            area = row['area']
            geom = row['geometry']
    
            # Mask SST trends with the sea geometry
            masked_trends = trend_significance_ds['trend'].rio.clip([geom], drop=True)
            masked_significance = trend_significance_ds['significant'].rio.clip([geom], drop=True)
    
            # Clip cell_areas to the same extent as masked_trends
            cell_areas_clipped = xr.DataArray(
                cell_areas, 
                dims=['y', 'x'], 
                coords={'y': trend_significance_ds['y'], 'x': trend_significance_ds['x']}
            )
            
            # Set CRS for cell_areas_clipped to match the CRS of trend_significance_ds
            cell_areas_clipped = cell_areas_clipped.rio.write_crs('EPSG:4326')
    
            # Clip cell_areas to the same geometry
            cell_areas_clipped = cell_areas_clipped.rio.clip([geom], drop=True)
        
            # Compute the area-weighted trend
            weighted_trend = (masked_trends * cell_areas_clipped).sum(dim=('y', 'x')) / cell_areas_clipped.sum()
    
            # Compute the total area that is significant
            significant_masked_areas = (masked_significance * cell_areas_clipped).where(masked_significance, 0)
            total_significant_area = significant_masked_areas.sum(dim=('y', 'x')).item()
    
            # Calculate the percentage of the area that is significant
            total_area = cell_areas_clipped.sum()
            significant_area_percent = (total_significant_area / total_area) * 100
    
            # Calculate the area for biodiversity based on the mask
            area_biodiversity = ((masked_significance * cell_areas_clipped) * masked_data_interp).sum(dim=['x', 'y']).values
    
            # Store the result
            area_weighted_trends.append({
                'Region_Name': region_name,
                'geometry': geom,
                'Weighted_Trend': weighted_trend.item(),
                'Sea_Area': area,
                'Significant_Area': area*total_significant_area/total_area.item(),
                'Significant_Area_Percent': 100*total_significant_area/total_area.item(),
                'Biodiversity_Area': area*area_biodiversity[0]/total_area.item(),
                'Biodiversity_Area_Percent': 100*area_biodiversity[0]/total_area.item(),
            })
        except Exception as e:
            print(e)

    # Convert the results to a GeoDataFrame for easy viewing
    area_weighted_trends_gdf = gpd.GeoDataFrame(area_weighted_trends, crs=SEAS_DF.crs)
    return area_weighted_trends_gdf

def area_heatwave(temp_df, SEAS_DF=SEAS_DF):
    area_heatwave = []

    # Set CRS and rename dimensions and coordinates
    temp_df = temp_df.rio.write_crs("epsg:4326")
    temp_df = temp_df.rename({'latdim': 'y', 'londim': 'x'}).rename({'lat': 'y', 'lon': 'x'})  # Adjust based on your dimensions

    # Select heatwave categories >= 3 and aggregate over time
    temp_df = (temp_df['heatwave_category'] >= 3).any(dim='time')

    # Interpolate biodiversity priority areas to the same resolution as the climate data
    masked_data_interp = masked_data.interp(
        x=temp_df['x'],
        y=temp_df['y'],
        method='nearest'
    )

    # Calculate the area for each grid cell (assumes lat/lon grid)
    lat = temp_df['y'].values
    lon = temp_df['x'].values
    
    # Calculate grid cell area using Haversine formula or by approximation
    lat_rad = np.deg2rad(lat)
    lon_rad = np.deg2rad(lon)
    
    # Earth radius in kilometers
    R = 6371
    dlat = np.gradient(lat_rad)
    dlon = np.gradient(lon_rad)
    
    # Approximate area calculation
    cell_areas = (R**2 * np.outer(np.sin(dlat), dlon)) * np.cos(lat_rad[:, None])
    
    # Use tqdm to track progress through SEAS_DF.iterrows()
    for i, row in tqdm(SEAS_DF.iterrows(), total=len(SEAS_DF), desc="Processing Sea Areas"):
        try:
            region_name = row['NAME']
            area = row['area']
            geom = row['geometry']
    
            # Mask SST trends with the sea geometry
            masked_df = temp_df.rio.clip([geom], drop=True)
    
            # Clip cell_areas to the same extent as masked_df
            cell_areas_clipped = xr.DataArray(
                cell_areas, 
                dims=['y', 'x'], 
                coords={'y': temp_df['y'], 'x': temp_df['x']}
            )
            
            # Set CRS for cell_areas_clipped to match the CRS of trend_significance_ds
            cell_areas_clipped = cell_areas_clipped.rio.write_crs('EPSG:4326')
    
            # Clip cell_areas to the same geometry
            cell_areas_clipped = cell_areas_clipped.rio.clip([geom], drop=True)
        
            # Compute the total area that is impacted by a severe heatwave
            heatwave_area = (masked_df * cell_areas_clipped).sum(dim=('y', 'x')).compute()  # Compute to convert from Dask array
    
            # Calculate the area for biodiversity based on the mask
            area_biodiversity = ((masked_df * cell_areas_clipped) * masked_data_interp).sum(dim=['x', 'y']).compute()

            total_area = cell_areas_clipped.sum(dim=('y', 'x')).compute()  # Ensure computation
    
            # Extract values after computing
            heatwave_value = heatwave_area.item() if heatwave_area.size == 1 else heatwave_area.values[0]
            total_area_value = total_area.item() if total_area.size == 1 else total_area.values[0]
            area_biodiversity = area_biodiversity.item() if area_biodiversity.size == 1 else area_biodiversity.values[0]
    
            # Store the result
            area_heatwave.append({
                'Region_Name': region_name,
                'geometry': geom,
                'Heatwave_Area': area*heatwave_value/total_area_value,
                'Heatwave_Area_Percent': 100*heatwave_value/total_area_value,
                'Sea_Area': area,
                'Biodiversity_Area': area*area_biodiversity/total_area_value,
                'Biodiversity_Area_Percent': 100*area_biodiversity/total_area_value,
            })
        except Exception as e:
            print(f"Error processing {region_name}: {e}")

    # Convert the results to a GeoDataFrame for easy viewing
    area_heatwave_gdf = gpd.GeoDataFrame(area_heatwave, crs=SEAS_DF.crs)
    return area_heatwave_gdf
    

# Temperature

## Figure 1

<p align="center">
  <img src="Figs/climate_temperature_1.png" style="width:50%;">
</p>

In [ ]:
ocean_data = pd.read_csv("../Data/GISTEMP_SST.csv") # Data downloaded from GISS Surface Temperature Analysis (v4)
gmst_data = pd.read_csv("../Data/GMST_GISTEMP4.csv") # Data downloaded from GISS Surface Temperature Analysis (v4)

ocean_data['Ocean_Annual'] = ocean_data['Lowess(5)']
gmst_data['GMST_Annual'] = gmst_data['Lowess(5)']

temp_data = ocean_data.merge(gmst_data,on='Year')

# Calculate the mean of the 'No_Smoothing' column for the period 1880-1900
base_period = temp_data[(temp_data['Year'] >= 1880) & (temp_data['Year'] <= 1900)]
mean_sst_base_period = base_period['Ocean_Annual'].mean()
mean_gmst_base_period = base_period['GMST_Annual'].mean()

# Update the 'No_Smoothing' column to be anomalies relative to the period 1880-1900
temp_data['Ocean_Annual'] = temp_data['Ocean_Annual'] - mean_sst_base_period
temp_data['GMST_Annual'] = temp_data['GMST_Annual'] - mean_gmst_base_period

# Perform linear regression to find the slope and intercept
slope, intercept, _, _, _ = linregress(temp_data['Year'], temp_data['Ocean_Annual'])

# Calculate the trend line (y = mx + b) for each time point
temp_data['ocean_trend'] = intercept + slope * temp_data['Year']

# Perform linear regression to find the slope and intercept
slope, intercept, _, _, _ = linregress(temp_data['Year'], temp_data['GMST_Annual'])

# Calculate the trend line (y = mx + b) for each time point
temp_data['gmst_trend'] = intercept + slope * temp_data['Year']
temp_data['paris_goal'] = 1.5

# Save out as JSON
temp_data[['Year','Ocean_Annual','ocean_trend','GMST_Annual','gmst_trend','paris_goal']].to_json(
    "../Data/Figure_1_temperature.json", 
    orient="records",  # list of dicts (one per row)
    indent=2           # pretty print
)


# Display the updated DataFrame
temp_data[['Year','Ocean_Annual','ocean_trend','GMST_Annual','gmst_trend','paris_goal']]


## Figure 2

<p align="center">
  <img src="Figs/climate_temperature_2.png" style="width:50%;">
</p>

In [ ]:
import numpy as np
import xarray as xr
import rioxarray

trends_df = xr.open_dataset("../Data/global_omi_tempsal_sst_trend_19932021_P20220331.nc")

# --------------------
# CONFIG
# --------------------
OUTPUT_TIF_8BIT   = "../Data/Figure_2_temperature.tif"
OUTPUT_TIF_FLOAT  = "../Data/sst_trend_float32.tif"
USE_PERCENTILES   = True
P_LOW, P_HIGH     = 2, 98
VMIN_FIXED, VMAX_FIXED = -0.5, 0.5  # if USE_PERCENTILES=False

# 1) Select the dataarray
da = trends_df["sst_trends"]
if "time" in da.dims:
    da = da.mean(dim="time")

# 2) Rename to x/y if needed
rename_map = {}
if "lat" in da.dims or "lat" in da.coords: rename_map["lat"] = "y"
if "latitude" in da.dims or "latitude" in da.coords: rename_map["latitude"] = "y"
if "lon" in da.dims or "lon" in da.coords: rename_map["lon"] = "x"
if "longitude" in da.dims or "longitude" in da.coords: rename_map["longitude"] = "x"
if rename_map:
    da = da.rename(rename_map)

# 3) Ensure y is north→south (descending)
if da["y"].values[0] < da["y"].values[-1]:
    da = da.sortby("y", ascending=False)

# 4) Register spatial dims & CRS
da = da.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)
if da.rio.crs is None:
    da = da.rio.write_crs("EPSG:4326", inplace=False)

# 5) Prepare data and scaling
data = da.data.astype(np.float32)
valid = np.isfinite(data)

if USE_PERCENTILES:
    vmin = float(np.nanpercentile(data, P_LOW))
    vmax = float(np.nanpercentile(data, P_HIGH))
else:
    vmin, vmax = float(VMIN_FIXED), float(VMAX_FIXED)

if not np.isfinite(vmin) or not np.isfinite(vmax) or vmin >= vmax:
    raise ValueError(f"Bad scaling range: vmin={vmin}, vmax={vmax}")

# 6) Build the 8-bit layer (reserve 0 for NoData)
scaled = np.zeros_like(data, dtype=np.uint8)  # 0 = NoData
scaled_valid = (np.clip((data[valid] - vmin) / (vmax - vmin), 0.0, 1.0) * 254 + 1).astype(np.uint8)
scaled[valid] = scaled_valid

da8 = da.copy(data=scaled)

# --- CRITICAL: clear CF encodings that carry a massive _FillValue ---
da8.encoding.pop("_FillValue", None)
da8.encoding.pop("missing_value", None)
# (optional) also clear scale/offset if present
da8.encoding.pop("scale_factor", None)
da8.encoding.pop("add_offset", None)

# Set nodata appropriate for uint8 (0)
da8 = da8.rio.write_nodata(0, encoded=False, inplace=False)

# 7a) Write 8-bit GeoTIFF
da8.rio.to_raster(
    OUTPUT_TIF_8BIT,
    dtype="uint8",
    compress="LZW",
    tiled=True,
    blockxsize=256,
    blockysize=256,
)

# 7b) Optional: write float32 with native values
daf = da.where(np.isfinite(da))
# Clear encodings here too
daf.encoding.pop("_FillValue", None)
daf.encoding.pop("missing_value", None)
daf.encoding.pop("scale_factor", None)
daf.encoding.pop("add_offset", None)

# Use NaN as nodata for float32 (supported by rasterio/GTiff)
daf = daf.rio.write_nodata(np.nan, encoded=False, inplace=False)

daf.rio.to_raster(
    OUTPUT_TIF_FLOAT,
    dtype="float32",
    compress="LZW",
    tiled=True,
    blockxsize=256,
    blockysize=256,
)

print(
    f"Wrote {OUTPUT_TIF_8BIT} (uint8; 0=NoData, 1–255=data) and {OUTPUT_TIF_FLOAT} (float32). "
    f"Scale for 8-bit: vmin={vmin:.4f}, vmax={vmax:.4f} (units of sst_trends)."
)


## Figure 3

<p align="center">
  <img src="Figs/climate_temperature_3.png" style="width:50%;">
</p>

**This figure calculates the total area for each major sea region globally. It also calculates 1) the area for each sea region impacted by a severe marine heatwave in the year 2023 as well as 2) the area for each sea that both has priority biodiversity areas AND is impacted by a severe marine heatwave.**

In [ ]:
# Year 2024 Heatwave data downloaded from NOAA's Coral Reef Watch https://coralreefwatch.noaa.gov/product/marine_heatwave/
import xarray as xr
import glob

files = sorted(glob.glob("../Data/2023/*.nc"))

temp_df = xr.open_mfdataset("../Data/2023/*.nc")

area_df = area_heatwave(temp_df)

# Save the GeoDataFrame to a GeoJSON file
area_df.to_file("../Data/Figure_3_temperature.geojson",driver="GeoJSON")

## Figure 4

<p align="center">
  <img src="Figs/climate_temperature_4.png" style="width:50%;">
</p>

$CO_2$ data retrieved from [Lan, X., P. Tans, & K.W. Thoning (2025). Trends in globally-averaged CO₂ determined from NOAA Global Monitoring Laboratory measurements (Version 2025-11) NOAA Global Monitoring Laboratory. https://doi.org/10.15138/9N0H-ZH07](https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_annmean_mlo.txt). Temperature data retrieved from GISS Surface Temperature Analysis (v4).

## Figure 5

<p align="center">
  <img src="Figs/climate_temperature_5.png" style="width:50%;">
</p>

Data were retrieved from [Hughes, T. P., et al. (2018). Spatial and temporal patterns of mass bleaching of corals in the Anthropocene. Science. – processed by Our World in Data](https://ourworldindata.org/grapher/coral-bleaching-events).

## Figure 6

<p align="center">
  <img src="Figs/climate_temperature_6.png" style="width:50%;">
</p>

In [ ]:
#!/usr/bin/env python3
import xarray as xr
import rioxarray  # pip install rioxarray rasterio

# --- Load and compute heatwave days ---
ds = xr.open_mfdataset("../Data/2023/*.nc", combine="by_coords")

# If you already have temp_df, you can instead do:
# hw_count = temp_df

hw_count = (ds["heatwave_category"] >= 1).sum(dim="time")
hw_count = hw_count.astype("float32")

# --- Rechunk along spatial dims for quantile ---
# Identify spatial dimensions (everything except 'time')
spatial_dims = [d for d in hw_count.dims if d != "time"]

# Make each spatial dimension a single chunk
hw_q = hw_count.chunk({d: -1 for d in spatial_dims})

# --- Compute 2nd and 98th percentiles over space ---
q = hw_q.quantile([0.02, 0.98], dim=spatial_dims, skipna=True).compute()

p2 = float(q.sel(quantile=0.02).values)
p98 = float(q.sel(quantile=0.98).values)

print(f"2nd percentile: {p2}, 98th percentile: {p98}")

if p98 <= p2:
    raise ValueError("98th percentile is not greater than 2nd percentile; cannot scale safely.")

# --- Scale to 0–255 using the 2nd and 98th percentiles ---
scaled = (hw_count - p2) / (p98 - p2) * 255.0
scaled = scaled.clip(0, 255)
scaled = scaled.fillna(0)

# Round and cast to uint8
scaled_uint8 = scaled.round().astype("uint8")

# --- Set spatial dims and CRS for Mapbox ---
# Try to guess your x/y dimension names
cands_x = ["lon", "longitude", "x", "londim"]
cands_y = ["lat", "latitude", "y", "latdim"]

x_dim = next(d for d in cands_x if d in scaled_uint8.dims)
y_dim = next(d for d in cands_y if d in scaled_uint8.dims)

scaled_uint8 = scaled_uint8.rio.set_spatial_dims(x_dim=x_dim, y_dim=y_dim, inplace=False)

# Ensure WGS84
if not scaled_uint8.rio.crs:
    scaled_uint8 = scaled_uint8.rio.write_crs("EPSG:4326")

# Optional: use 0 as nodata for transparency in Mapbox
scaled_uint8 = scaled_uint8.rio.write_nodata(0)

# --- Save as GeoTIFF ---
out_path = "Figure_6_temperature.tif"
scaled_uint8.rio.to_raster(out_path, dtype="uint8")

print(f"Saved GeoTIFF: {out_path}")


## Figure 7

<p align="center">
  <img src="Figs/climate_temperature_7.png" style="width:50%;">
</p>

Data were retrieved from [Jones et al. (2024) – with major processing by Our World in Data](https://ourworldindata.org/co2-and-greenhouse-gas-emissions#explore-data-on-co2-and-greenhouse-gas-emissions).